# This notebook check available phenotypes from AWIGEN

In [1]:
import pandas as pd

In [2]:
pheno_folder = "/expanse/protected/gymreklab-dbgap/mount/H3Africa/DS_I_Africa_project/from_ilifu/awigen_dataset/phenotypes"

In [75]:
pheno_df = pd.read_csv(pheno_folder+"/EGAF00004691444/EGA_dataset_v0_1.csv")
pheno_df.columns = [i.strip(" ") for i in pheno_df.columns.tolist()]
pheno_df["study_id"] = pheno_df["study_id"].apply(lambda x: x.strip(" "))
pheno_df["country_qc"] = pheno_df["study_id"].apply(lambda x: x.strip(" "))

## Find samples that overlap with the VCF files

In [72]:
ind_df = pd.read_csv("/expanse/protected/gymreklab-dbgap/mount/H3Africa/DS_I_Africa_project/from_ilifu/results/GWAS/PCs/calculate_pc/ind",
                     header=None,
                     usecols=[0,1],
                     sep="\s+", 
                     names=["study_id","sex"])
ind_df = ind_df.merge(pheno_df[["study_id","country_qc"]],on="study_id",how="left")
# fill those missing country one with "no_label"
ind_df.country_qc.fillna("no_label",inplace=True)
ind_df.to_csv("/expanse/protected/gymreklab-dbgap/mount/H3Africa/DS_I_Africa_project/from_ilifu/results/GWAS/PCs/calculate_pc/ind_country",
              sep="\t",
              header=None,
              index=None)

In [81]:
pheno_df[pheno_df["sex"].isnull()]

,study_id,cohort_id_c,site,age,sex,country_qc,number_of_sons_qc,number_of_daughters_qc,number_of_children_c,number_of_pregnancies_qc,...,friedewald_ldl_c_c_qc,cholesterol_1_qc,non_hdl_c_c_qc,triglycerides_qc,ur_creatinine_qc,ur_albumin_qc,ur_protein_qc,acr_qc,egfr_c_qc,ckd_c_qc
12032,(12032 rows),NaN,NaN,NaN,NaN,(12032 rows),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
pheno_df[[i  for i in pheno_df.columns.tolist() if "ldl" in i]][" ldl_qc "]#.isnull().sum()

0          0.63
1          3.34
2          2.55
3          3.96
4          2.13
          ...  
12028      3.79
12029      2.68
12030      2.73
12031   -222.00
12032       NaN
Name:  ldl_qc , Length: 12033, dtype: float64

## Load the definition of all columns

In [83]:
phnoe_excel = pd.ExcelFile(pheno_folder+"/EGAF00004691445/V2.0_AWI-Gen_Code_Book-EGA_Variables.xlsx")

In [84]:
phnoe_excel.sheet_names

['Main',
 '1. Demographic Information',
 '2. Behaviour_Lifestyle',
 '3. Health History',
 '4. Family History',
 '5. Exposure',
 '6. Infection History',
 '7. Cardiovascular Disease Risk',
 '8. Measurements & Calculations']

In [ ]:
df = phnoe_excel.parse('8. Measurements & Calculations',skiprows=2)

In [85]:
df = phnoe_excel.parse('1. Demographic Information',skiprows=2)

In [30]:
df.loc[21]["Categorical Description"]

'-999 - Missing\n-111 - Out-of-range (lower bound)\n-222 - Out-of-range (up bound)'